### To Do / License





- This should take a text description of a campaign and Gemini should help determine the channels
  - Gemini must explain its logic

- This also should take a video (not sure if you need to create one in advance) and then Gemini should watch the video and select the customer channels
  - Gemini must explain its logic

- This will also need to allocate the budget to the different segments using Gemini

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

Author: Adam Paternostro

### Pip installs

In [ ]:
# To read/write to/from Kafka
import sys

# https://PLACEHOLDER.com/index.html

# For better performance and production, deploy to Vertex AI endpoint with GPU
# This takes about 5 minutes to install and you will need to reset your runtime
#!{sys.executable} -m pip install timesfm

### Initialize

In [ ]:
from PIL import Image
from IPython.display import HTML
from IPython.display import Audio
from functools import reduce
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "us"
region = "us-central1"
location = "us-central1"
storage_account = "data-analytics-preview"
public_storage_storage_account = "data-analytics-golden-demo"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### Helper Methods

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
def GeminiLLM_Multimodal(multimodal_prompt_list, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": multimodal_prompt_list
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Helper Functions

In [ ]:
# prompt: python to delete a file even if it does not exist

def delete_file(filename):
  try:
    os.remove(filename)
    print(f"File '{filename}' deleted successfully.")
  except FileNotFoundError:
    print(f"File '{filename}' not found.")

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  #print(json_formatted_str)
  return json_formatted_str

In [ ]:
# prompt: python code to download a pdf from the internet

import requests

def download_http_file(url, filename):
  """Downloads a PDF file from a given URL.

  Args:
      url: The URL of the PDF file to download.
      filename: The name to save the downloaded PDF file as.
  """
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(filename, 'wb') as f:
      f.write(response.content)

    print(f"PDF downloaded successfully to {filename}")

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while downloading the PDF: {e}")

# Example usage:


In [ ]:
def download_from_gcs(filename, gcs_storage_bucket, gcs_storage_path):
  # prompt: Write python code to download a blob from a gcs bucket.  do not use the requests method

  from google.cloud import storage

  # The ID of your GCS object
  object_name = gcs_storage_path + filename

  # The path to which the file should be downloaded
  destination_file_name = filename

  storage_client = storage.Client()

  bucket = storage_client.bucket(gcs_storage_bucket)

  # Construct a client side representation of a blob.
  # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
  # any content from Google Cloud Storage. As we don't need additional data,
  # using `Bucket.blob` is preferred here.
  blob = bucket.blob(object_name)
  blob.download_to_filename(destination_file_name)

  print(
      "Downloaded storage object {} from bucket {} to local file {}.".format(
          object_name, gcs_storage_bucket, destination_file_name
      )
  )

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

### Target Channels

In [ ]:
# Download our sample Chocolate A.I. Ad and upload to our project storage account
download_from_gcs("mocha-magic-full-ad.mp4", "data-analytics-golden-demo", "chocolate-ai/v2/DB-GMA-Campaign-Assets-Video-Text2Video/Mocha-Magic/")
copy_file_to_gcs("mocha-magic-full-ad.mp4", storage_account, f"chocolate-ai/DB-GMA-Create-Campaign-Target-Channels/mocha-magic-channel-{formatted_date}/mocha-magic-full-ad.mp4")

In [ ]:
# Download our sample Chocolate A.I. Ad and upload to our project storage account
download_from_gcs("youtube-short.mp4", "data-analytics-golden-demo", "chocolate-ai/v2/DB-GMA-Campaign-Assets-Video-Create-Shorts/Mocha-Magic/")
copy_file_to_gcs("youtube-short.mp4", storage_account, f"chocolate-ai/DB-GMA-Create-Campaign-Target-Channels/mocha-magic-channel-{formatted_date}/mocha-magic-short.mp4")

In [ ]:
# Prompt: You work at a marketing company and need a list of advertising channels. Create a list channels along with a description for each.
# Output as a Python Dictionary
# Include the following fields: ...

advertising_channels = {
    "Digital Advertising": {
        "Search Engine Marketing (SEM)": {
            "description": "Paid advertising on search engine results pages (SERPs) like Google, Bing, etc.",
            "vendors": ["Google Ads", "Bing Ads"],
            "pros": ["Highly targeted", "Measurable", "Quick results"],
            "cons": ["Can be expensive", "Requires ongoing management"]
        },
        "Search Engine Optimization (SEO)": {
            "description": "Optimizing your website to rank higher organically in search engine results.",
            "vendors": ["SEMrush", "Ahrefs", "Moz" ,"Google Search Console"], # Tools rather than vendors
            "pros": ["Long-term ROI", "Increased credibility and brand authority"],
            "cons": ["Requires time and effort", "Results are not immediate"]
        },
        "Social Media Advertising": {
            "description": "Paid ad campaigns on platforms like Facebook, Instagram, Twitter, LinkedIn, etc.",
            "vendors": ["YouTube Shorts", "Facebook Ads", "Instagram Ads", "Twitter Ads", "LinkedIn Ads", "TikTok Ads", "Pinterest Ads"],
            "pros": ["Precise audience targeting", "Diverse ad formats", "Relatively affordable"],
            "cons": ["Can be perceived as intrusive", "Requires platform expertise"]
        },
        "RoboTaxi Advertising": {
            "description": "Show videos to customers that are near a specific geospatial coordinate.",
            "vendors": ["RoboTaxi"],
            "pros": [
                "Hyperlocal Targeting", # Reaching customers precisely where they are
                "Captive Audience",  # Passengers have limited distractions during rides
                "Measurable Results", # Track ad views, engagement, and even potential store visits
                "Novelty Factor", #  New and potentially engaging ad format
                "Integration with Ride Experience", # Ads can be tailored to destinations or journey times
            ],
            "cons": [
                "Privacy Concerns", # Targeting based on location data requires transparency and user consent
                "Limited Reach",  #  Reaches only those using robotaxi services
                "Potential for Intrusiveness", #  Striking a balance between advertising and passenger experience is crucial
                "Technical Challenges", #  Requires robust geofencing and ad delivery technology
                "Cost Considerations", #  Pricing models and potential ROI need to be established
            ]
        },
        "Display Advertising": {
            "description": "Visual ads on websites, apps, and social media platforms, often using banner ads or rich media.",
            "vendors": ["Google Display Network", "DoubleClick", "Adform", "The Trade Desk"],
            "pros": ["Broad reach", "Visual impact", "Various targeting options"],
            "cons": ["Can be ignored", "Ad blindness is a factor", "Can be expensive"]
        },
        "Video Advertising": {
            "description": "Ads displayed on platforms like YouTube, Vimeo, streaming services, and social media.",
            "vendors": ["YouTube Ads", "Vimeo Ads", "Amazon Advertising", "Hulu Ads"],
            "pros": ["Engaging format", "High completion rates", "Strong storytelling potential"],
            "cons": ["Higher production costs", "Ad skipping can be an issue"]
        },
        "Mobile Advertising": {
            "description": "Ads specifically targeted to mobile devices, through apps, mobile websites, and SMS.",
            "vendors": ["Google Mobile Ads", "Apple Search Ads", "InMobi", "AdColony"],
            "pros": ["Reaches a vast and engaged audience", "Highly personalizable"],
            "cons": ["Can be intrusive", "Ad fatigue is common", "Requires mobile optimization"]
        },
        "Email Marketing": {
            "description": "Sending targeted marketing messages directly to an audience's inbox.",
            "vendors": ["Mailchimp", "Campaign Monitor", "Constant Contact", "GetResponse"],
            "pros": ["Direct communication", "High ROI", "Trackable results"],
            "cons": ["Requires list building", "Can be flagged as spam", "Needs compelling content"]
        },
        "Affiliate Marketing": {
            "description": "Partnering with other businesses or individuals to promote your products or services for a commission.",
            "vendors": ["Amazon Associates", "ShareASale", "CJ Affiliate", "ClickBank"],
            "pros": ["Performance-based", "Access to new audiences", "Cost-effective"],
            "cons": ["Requires relationship building", "Can be complex to manage"]
        },
        "Influencer Marketing": {
            "description": "Collaborating with influential people on social media or other platforms to promote your brand.",
            "vendors": ["Influencer marketing platforms", "Talent agencies", "Direct outreach to influencers"], # Less specific vendors, more about approach
            "pros": ["Reaches a targeted audience", "Builds credibility and trust"],
            "cons": ["Can be expensive", "Requires careful vetting of influencers"]
        },
        "Native Advertising": {
            "description": "Ads that blend in with the surrounding content on a platform, aiming to be less intrusive.",
            "vendors": ["Outbrain", "Taboola", "Yahoo Gemini", "TripleLift"],
            "pros": ["High engagement rates", "Less ad fatigue", "Perceived as valuable content"],
            "cons": ["Can be deceptive if not executed ethically", "Requires careful planning"]
        }
    },
    "Traditional Advertising": {
        "Television Advertising": {
            "description": "Commercials aired during television programs.",
            "vendors": ["ITV", "Channel 4", "Sky", "London TV"],
            "pros": ["Broad reach", "High impact", "Strong emotional connection"],
            "cons": ["Expensive", "Declining viewership", "Difficult to target specific audiences"]
        },
        "Radio Advertising": {
            "description": "Ads broadcasted on radio stations.",
            "vendors": ["Global", "Bauer Media", "Wireless Group", "Communicorp"], # Major radio groups in London
            "pros": ["Local targeting", "Affordable", "Can reach people on the go"],
            "cons": ["Audio-only format", "Can be easily tuned out", "Difficult to track ROI"]
        },
        "Print Advertising": {
            "description": "Ads placed in newspapers, magazines, brochures, etc.",
            "vendors": ["Evening Standard", "Metro", "Time Out London", "The Londonist"], # Popular London publications
            "pros": ["Tangible format", "Targeted reach for specific audiences", "Longer shelf life"],
            "cons": ["Declining readership", "Long lead times", "Difficult to track effectiveness"]
        },
        "Out-of-Home Advertising (OOH)": {
            "description": "Ads placed in public spaces, such as billboards, transit ads, street furniture, etc.",
            "vendors": ["JCDecaux", "Global", "Clear Channel", "ES Digital"], # Major OOH companies in London
            "pros": ["High visibility", "Constant exposure", "Geographically targeted"],
            "cons": ["Difficult to measure effectiveness", "Limited messaging space", "Can be costly"]
        },
        "Direct Mail Marketing": {
            "description": "Sending physical marketing materials, like postcards, flyers, catalogs, directly to potential customers.",
            "vendors": ["Royal Mail MarketReach", "Printing companies", "Mailing houses"], # More about service providers
            "pros": ["Tangible and personalized", "Can be highly targeted"],
            "cons": ["High production and postage costs", "Often perceived as junk mail"]
        }
    },
    "Emerging Advertising Channels": {
        "Augmented Reality (AR) & Virtual Reality (VR) Advertising": {
            "description": "Interactive and immersive ad experiences.",
            "vendors": ["Snapchat", "Facebook (Spark AR)", "Zappar", "Blippar"] # Platforms and AR development companies
        },
        "Connected TV (CTV) Advertising": {
            "description": "Targeted ads on streaming services accessed through internet-connected TVs.",
            "vendors": ["Hulu", "Amazon Advertising", "Roku", "Samsung Ads"]
        },
        "Digital Audio Advertising": {
            "description": "Ads on music streaming platforms, podcasts, and digital radio.",
            "vendors": ["Spotify", "Apple Music", "Pandora", "Acast"],
        },
        "Programmatic Advertising": {
            "description": "Using technology to automate the buying and selling of ad space in real-time.",
            "vendors": ["Google Display & Video 360", "The Trade Desk", "Adobe Advertising Cloud", "Amazon DSP"]
        }
    }
}

In [ ]:
# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  [{
#    "channel" : "text",
#    "channel-video": "text",
#    "channel-budget": 1.5,
#    "channel-vendors" : ["text"],
#    "channel-ranking" : 1,
#    "channel-pros" : ["text"],
#    "channel-cons" : ["text"],
#    "explanation" : "text"
#  }]
response_schema = {
  "type": "array",
  "items": {
    "type": "object",
    "required": [
      "channel",
      "channel-video",
      "channel-budget",
      "channel-vendors",
      "channel-ranking",
      "channel-pros",
      "channel-cons",
      "explanation"
    ],
    "properties": {
      "channel": {
        "type": "string"
      },
      "channel-video": {
        "type": "string"
      },
      "channel-budget": {
        "type": "number"
      },
      "channel-vendors": {
        "type": "array",
        "items": {
          "type": "string"
        }
      },
      "channel-ranking": {
        "type": "integer"
      },
      "channel-pros": {
        "type": "array",
        "items": {
          "type": "string"
        }
      },
      "channel-cons": {
        "type": "array",
        "items": {
          "type": "string"
        }
      },
      "explanation": {
        "type": "string"
      }
    }
  }
}

prompt = f"""You work at Chocolate A.I., a coffee truck company based in London.
You are running a campaign called "Mocha Magic" and have a video that shows the new ad.
I have also included a "short" video that can be used.
I need you to recommend the top 2 channels to use for the new ad.
I have a $4,000 budget, allocate the budget to the top 5 channels.  Make sure you explain why in the explaination field.

Output the following JSON in OpenAPI 3.0 schema object:
channel: the name of the channel
channel-video: "mocha-magic-full-ad.mp4" or "video-short.mp4"
channel-vendors: the names of the vendors that the channel is targeted for
channel-ranking: the ranking of the channel.  The ranking is from 1 to 2.  The lower the ranking, the better the channel.
channel-pros: the pros of the channel
channel-cons: the cons of the channel
explanation: a brief explanation of the channel

<AdvertisingChannels>
{advertising_channels}
</AdvertisingChannels>
"""

multimodal_prompt_list = [
    { "text": prompt },
    { "fileData": {  "mimeType": "video/mp4", "fileUri": f"gs://data-analytics-preview/chocolate-ai/DB-GMA-Create-Campaign-Target-Channels/mocha-magic-channel-{formatted_date}/mocha-magic-full-ad.mp4" } },
    { "fileData": {  "mimeType": "video/mp4", "fileUri": f"gs://data-analytics-preview/chocolate-ai/DB-GMA-Create-Campaign-Target-Channels/mocha-magic-channel-{formatted_date}/mocha-magic-short.mp4" } },
  ]

channel_response = GeminiLLM_Multimodal(multimodal_prompt_list, response_schema=response_schema)

In [ ]:
result = PrettyPrintJson(channel_response)
print(result)

```
   "Instagram Ads"
    ],
    "channel-video": "mocha-magic-full-ad.mp4",
    "explanation": "Social Media is a great way to reach your target audience. You can use platforms like Facebook and Instagram to target people in London who are interested in coffee or food trucks.  The full ad is best suited for the demographic that use these platforms."
  },
  {
    "channel": "RoboTaxi Advertising",
    "channel-budget": 2000.0,
    "channel-cons": [
      "Privacy Concerns",
      "Limited Reach",
      "Potential for Intrusiveness",
      "Technical Challenges",
      "Cost Considerations"
    ],
    "channel-pros": [
      "Hyperlocal Targeting",
      "Captive Audience",
      "Measurable Results",
      "Novelty Factor",
      "Integration with Ride Experience"
    ],
    "channel-ranking": 2,
    "channel-vendors": [
      "RoboTaxi"
    ],
    "channel-video": "video-short.mp4",
    "explanation": "RoboTaxi advertising allows you to target customers that are near your coffee truck in real-time.  Since this is a novel technology, using a shorter video will allow for more views."
  },
  {
    "channel": "Search Engine Marketing (SEM)",
    "channel-budget": 0.0,
    "channel-cons": [
      "Can be expensive",
      "Requires ongoing management"
    ],
    "channel-pros": [
      "Highly targeted",
      "Measurable",
      "Quick results"
    ],
    "channel-ranking": 3,
    "channel-vendors": [
      "Google Ads",
      "Bing Ads"
    ],
    "channel-video": "mocha-magic-full-ad.mp4",
    "explanation": "SEM can be effective but may quickly eat your budget, consider for future campaigns."
  },
  {
    "channel": "Mobile Advertising",
    "channel-budget": 0.0,
    "channel-cons": [
      "Can be intrusive",
      "Ad fatigue is common",
      "Requires mobile optimization"
    ],
    "channel-pros": [
      "Reaches a vast and engaged audience",
      "Highly personalizable"
    ],
    "channel-ranking": 4,
    "channel-vendors": [
      "Google Mobile Ads",
      "Apple Search Ads"
    ],
    "channel-video": "mocha-magic-full-ad.mp4",
    "explanation": "Mobile advertising can be effective, but it is important to make sure that your ads are not intrusive and that they are targeted to the right audience.  Due to budget constraints, we will not use this channel at this time."
  },
  {
    "channel": "Video Advertising",
    "channel-budget": 0.0,
    "channel-cons": [
      "Higher production costs",
      "Ad skipping can be an issue"
    ],
    "channel-pros": [
      "Engaging format",
      "High completion rates",
      "Strong storytelling potential"
    ],
    "channel-ranking": 5,
    "channel-vendors": [
      "YouTube Ads"
    ],
    "channel-video": "mocha-magic-full-ad.mp4",
    "explanation": "Video advertising can be a great way to reach a large audience on platforms like YouTube.  Due to budget constraints, we will not use this channel at this time."
  }
]
```

### TimesFM

In [ ]:
import timesfm

In [ ]:
timesfm_backend = "cpu" # "cpu": "cpu", "gpu": "cuda", "tpu": ""

from jax._src import config
config.update("jax_platforms", timesfm_backend)

model = timesfm.TimesFm(
    context_len=512,
    horizon_len=61, # Predict next 61 days, this could be 128 without requiring compute (129 would be a step up).  This is more of the max horizon len.
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="cpu",
)

# This can produce "ERROR:absl:For checkpoint version > 1.0, we require users to provide", you can ignore that
model.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

### Predict Campaign Sales


**Scenerios**
```
0. Baseline - No campaign
1. Split the budget   0% / 100%   RoboTaxi / YouTube Shorts
2. Split the budget  25% /  75%   RoboTaxi / YouTube Shorts
3. Split the budget  50% /  50%   RoboTaxi / YouTube Shorts
4. Split the budget  75% /  25%   RoboTaxi / YouTube Shorts
5. Split the budget   0% / 100%   YouTube Shorts / RoboTaxi
6. Split the budget  25% /  75%   YouTube Shorts / RoboTaxi
7. Split the budget  50% /  50%   YouTube Shorts / RoboTaxi
8. Split the budget  75% /  25%   YouTube Shorts / RoboTaxi
```

In [ ]:
# predict the next 2 months "Nov/Dev 2024"
from datetime import date, timedelta
start_date = date(2024, 11, 1)
end_date   = date(2024, 12, 31)
delta = end_date - start_date
list_predicted_dates = []

context_robotaxi_cumulative_campaign_spend = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0]
context_youtube_shorts_cumulative_campaign_spend =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
context_sales_input = [2300.00,2302.30,2306.90,2306.90,2318.44,2327.71,2332.37,2332.37,2344.03,2351.06,2355.76,2365.19,2374.65,2386.52,2398.45,2528.45,2591.24,2664.76,2738.95,2799.81,2872.12,2933.23,3009.31,3562.04,3695.93,3834.40,3714.42,3466.79,3225.60,2986.41,2737.54,2715.37,2703.91,2684.23,2686.92,2521.18,2528.75,2538.86,2541.40,2546.48,2559.21,2559.21,2559.21,2569.45,2574.59,2577.16,2584.90,2592.65,2592.65,2595.24,2608.22,2671.03,2749.58,2830.97,2921.25,3003.20,3082.23,3173.91,3253.26,3332.61,3418.78,3505.28,3595.70,3679.28,3774.30,3866.17,3958.56,4047.42,4144.88,4247.29,4341.95,2741.71,2744.45,2744.45,2749.94,2758.19,2763.71,2774.76,2785.86,2794.22,2802.60,2805.40,2805.40,2813.82,2827.89,2827.89,2833.55,2847.71,2859.10,2861.96,2861.96,2867.69,2870.55,2882.04,2884.92,2899.34,2910.94,2913.85,2981.05,3068.72,3156.40,3250.57,3351.77,3453.74,3549.38,3649.03,3752.98,3857.71,3947.42,4037.14,4139.23,4237.67,4340.82,4453.41,4562.47,4663.03,4754.46,4870.12,3072.16,3081.38,3093.70,3106.08,3112.29,3112.29,3127.85,3130.98,3134.11,3143.51,3143.51,3152.94,3152.94,3317.21,3402.98,3482.12,3571.94,3654.97,3753.10,3832.95,3920.63,4654.67,4834.44,5020.56,4878.04,4566.50,4261.52,3937.65,3616.73,3594.60,3561.62,3539.23,3549.85,3330.89,3334.22,3340.89,3347.57,3347.57,3354.26,3360.97,3371.05,3377.80,3384.55,3394.71,3404.89,3415.10,3418.52,3418.52,3418.52,3435.61,3449.35,3459.70,3463.16,3470.09,3483.97,3501.39,3511.89,3522.43,3533.00,3540.06,3550.68,3564.88,3564.88,3564.88,3564.88,3572.01,3582.73,3672.66,3792.02,3900.36,4008.70,4125.28,4250.78,4377.21,4500.10,4619.08,4743.25,4868.11,4988.71,5114.87,5236.51,5347.93,5475.72,5598.65,5739.17,5857.56,5994.08,3773.63,3792.50,3807.67,3822.90,3834.37,3842.04,3861.25,3865.11,3880.57,3892.21,3896.11,3915.59,3923.42,3943.03,3958.81,3970.68,3986.57,4006.50,4010.51,4022.54,4042.65,4062.86,4083.18,4095.43,4111.81,4115.92,4132.38,4153.05,4157.20,4169.67,4173.84,4190.54,4207.30,4207.30,4228.33,4232.56,4236.79,4245.27,4266.49,4270.76,4279.30,4292.14,4524.77,4632.51,4754.46,4872.24,4980.51,5088.79,5207.45,5326.57,6330.14,6548.42,6787.00,6568.07,6130.20,5703.71,5275.49,4860.05,4830.31,4790.78,4751.17,4770.18,4475.94,4493.85,4507.33,4529.86,4552.51,4575.28,4584.43,4593.59,4593.59,4616.56,4630.41,4635.04,4653.58,4658.24,4658.24,4667.55,4686.22,4704.97,4728.49,4737.95,4832.71,4979.82,5122.10,5274.91,5439.15,5604.61,5771.31,5927.43,6084.14,6235.23,6405.76,6564.45,6710.32,6890.48,7072.27,7234.05,7418.59,7597.23,7761.69,7934.69,5000.35,5010.35,5035.40,5055.54,5075.77,5085.92,5096.09,5372.30,5511.21,5639.38,5773.31,5919.31,6072.19,6226.19,6387.68,7591.17,7876.49,8171.60,7923.81,7395.56,6887.91,6364.43,5863.23,5821.55,5779.67,5749.06,5766.31,5416.02,5416.02,5426.85,5432.28,5459.44,5486.74,5514.17,5530.71,5558.37,5575.04,5586.19,5591.78,5608.55,5608.55,5630.99,5630.99,5647.88,5647.88,5653.53,5676.14,5687.50,5715.93,5738.80,5738.80,5767.49,5796.33,5813.72,5813.72,5842.79,5842.79,5860.31,5866.17,5872.04,5883.78,5901.44,5930.94,5936.87,5960.62,5984.46,5996.43,6008.43,6026.45,6346.76,6510.87,6662.28,6834.14,7020.94,7180.74,7362.86,7553.83,8985.98,9295.84,9634.51,9361.02,8780.64,8161.60,7563.92,6947.46,6898.07,6834.79,6805.36,6812.16,6366.51,6366.51,6391.97,6411.15,6430.38,6462.54,6481.92,6514.33,6527.36,6546.94,6573.13,6586.28,6599.45,6632.45,6652.34,6659.00,6678.97,6685.65,6692.34,6692.34,6725.80,6732.53,6759.46,6779.73,6786.51,6943.01,7175.81,7388.21,7631.41,7869.01,8100.32,8316.33,8524.23,8758.34,9002.74,9239.74,9478.08,9708.09,9919.13,10150.44,10403.35,10626.28,10860.48,11073.43,11308.96,7141.00,7155.28,7191.06,7219.82,7219.82,7241.48,7241.48,7241.48,7241.48,7277.69,7306.80,7306.80,7321.42,7321.42,7343.38,7350.72,7372.78,7409.64,7819.02,8005.19,8224.12,8419.45,8649.58,8872.96,9107.06,9343.27,11092.56,11520.96,11928.80,11590.18,10849.95,10125.33,9393.19,8636.26,8557.75,8479.24,8417.53,8451.20,7906.21,7937.84,7953.71,7969.62,7985.56,8001.53,8009.53,8049.58,8065.68]
context_campaign_type = []

for i in range(len(context_sales_input)):
  if context_robotaxi_cumulative_campaign_spend[i] > 0:
    context_campaign_type.append("robotaxi")
  elif context_youtube_shorts_cumulative_campaign_spend[i] == 0:
    context_campaign_type.append("youtube_shorts")
  else:
    context_campaign_type.append("none")

# 0. Baseline - No campaign
scenerio_0_campaign_type = context_campaign_type.copy()
scenerio_0_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_0_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 1. Split the budget   0% / 100%   RoboTaxi / YouTube Shorts
scenerio_1_campaign_type = context_campaign_type.copy()
scenerio_1_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_1_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 2. Split the budget  25% /  75%   RoboTaxi / YouTube Shorts
scenerio_2_campaign_type = context_campaign_type.copy()
scenerio_2_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_2_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 3. Split the budget  50% /  50%   RoboTaxi / YouTube Shorts
scenerio_3_campaign_type = context_campaign_type.copy()
scenerio_3_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_3_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 4. Split the budget  75% /  25%   RoboTaxi / YouTube Shorts
scenerio_4_campaign_type = context_campaign_type.copy()
scenerio_4_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_4_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 5. Split the budget   0% / 100%   YouTube Shorts / RoboTaxi
scenerio_5_campaign_type = context_campaign_type.copy()
scenerio_5_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_5_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 6. Split the budget  25% /  75%   YouTube Shorts / RoboTaxi
scenerio_6_campaign_type = context_campaign_type.copy()
scenerio_6_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_6_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 7. Split the budget  50% /  50%   YouTube Shorts / RoboTaxi
scenerio_7_campaign_type = context_campaign_type.copy()
scenerio_7_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_7_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# 8. Split the budget  75% /  25%   YouTube Shorts / RoboTaxi
scenerio_8_campaign_type = context_campaign_type.copy()
scenerio_8_robotaxi_horizon = context_robotaxi_cumulative_campaign_spend.copy()
scenerio_8_youtube_horizon = context_youtube_shorts_cumulative_campaign_spend.copy()

# Assuming we spend $100 a day (to kept this somewhat simple)
budget_0_days = 0
budget_100 = 40
budget_25 = 10
budget_50 = 20
budget_75 = 30

robotaxi_increase = 200
youtube_increase = 200

# Initial Base Price
scenerio_1_amount_youtube = 1000
scenerio_1_amount_robotaxi = robotaxi_increase
scenerio_2_amount_youtube = youtube_increase
scenerio_2_amount_robotaxi = robotaxi_increase
scenerio_3_amount_youtube = youtube_increase
scenerio_3_amount_robotaxi = robotaxi_increase
scenerio_4_amount_youtube = youtube_increase
scenerio_4_amount_robotaxi = robotaxi_increase
scenerio_5_amount_youtube = youtube_increase
scenerio_5_amount_robotaxi = robotaxi_increase
scenerio_6_amount_youtube = youtube_increase
scenerio_6_amount_robotaxi = robotaxi_increase
scenerio_7_amount_youtube = youtube_increase
scenerio_7_amount_robotaxi = robotaxi_increase
scenerio_8_amount_youtube = youtube_increase
scenerio_8_amount_robotaxi = robotaxi_increase

for i in range(delta.days + 1):
  day = start_date + timedelta(days=i)
  list_predicted_dates.append(day)

  # 0. Baseline - No campaign
  scenerio_0_campaign_type.append("none")
  scenerio_0_robotaxi_horizon.append(0)
  scenerio_0_youtube_horizon.append(0)

  # 1. Split the budget   0% / 100%   RoboTaxi / YouTube Shorts (0 / 20 days)
  # High upfront cost so we can only do 15 days instead of 20
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 15):
    scenerio_1_campaign_type.append("youtube_shorts")
    scenerio_1_robotaxi_horizon.append(0)
    scenerio_1_youtube_horizon.append(scenerio_1_amount_youtube)
    scenerio_1_amount_youtube = scenerio_1_amount_youtube + youtube_increase
  else:
    scenerio_1_campaign_type.append("none")
    scenerio_1_robotaxi_horizon.append(0)
    scenerio_1_youtube_horizon.append(0)

  # 2. Split the budget  25% /  75%   RoboTaxi / YouTube Shorts (5 / 15 days)
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 15):
    scenerio_2_campaign_type.append("youtube_shorts")
    scenerio_2_robotaxi_horizon.append(0)
    scenerio_2_youtube_horizon.append(scenerio_2_amount_youtube)
    scenerio_2_amount_youtube = scenerio_2_amount_youtube + youtube_increase
  elif day >= date(2024, 12, 1) and day <= date(2024, 12, 5):
    scenerio_2_campaign_type.append("robotaxi")
    scenerio_2_robotaxi_horizon.append(scenerio_2_amount_robotaxi)
    scenerio_2_youtube_horizon.append(0)
    scenerio_2_amount_robotaxi = scenerio_2_amount_robotaxi + robotaxi_increase
  else:
    scenerio_2_campaign_type.append("none")
    scenerio_2_robotaxi_horizon.append(0)
    scenerio_2_youtube_horizon.append(0)

  # 3. Split the budget  50% /  50%   RoboTaxi / YouTube Shorts (10 / 10 days)
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 10):
    scenerio_3_campaign_type.append("youtube_shorts")
    scenerio_3_robotaxi_horizon.append(0)
    scenerio_3_youtube_horizon.append(scenerio_3_amount_youtube)
    scenerio_3_amount_youtube = scenerio_3_amount_youtube + youtube_increase
  elif day >= date(2024, 12, 1) and day <= date(2024, 12, 10):
    scenerio_3_campaign_type.append("robotaxi")
    scenerio_3_robotaxi_horizon.append(scenerio_3_amount_robotaxi)
    scenerio_3_youtube_horizon.append(0)
    scenerio_3_amount_robotaxi = scenerio_3_amount_robotaxi + robotaxi_increase
  else:
    scenerio_3_campaign_type.append("none")
    scenerio_3_robotaxi_horizon.append(0)
    scenerio_3_youtube_horizon.append(0)

  # 4. Split the budget  75% /  25%   RoboTaxi / YouTube Shorts (15 / 5 days)
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 5):
    scenerio_4_campaign_type.append("youtube_shorts")
    scenerio_4_robotaxi_horizon.append(0)
    scenerio_4_youtube_horizon.append(scenerio_4_amount_youtube)
    scenerio_4_amount_youtube = scenerio_4_amount_youtube + youtube_increase
  elif day >= date(2024, 12, 1) and day <= date(2024, 12, 15):
    scenerio_4_campaign_type.append("robotaxi")
    scenerio_4_robotaxi_horizon.append(scenerio_4_amount_robotaxi)
    scenerio_4_youtube_horizon.append(0)
    scenerio_4_amount_robotaxi = scenerio_4_amount_robotaxi + robotaxi_increase
  else:
    scenerio_4_campaign_type.append("none")
    scenerio_4_robotaxi_horizon.append(0)
    scenerio_4_youtube_horizon.append(0)

  # 5. Split the budget   0% / 100%   YouTube Shorts / RoboTaxi (0 / 20 days)
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 20):
    scenerio_5_campaign_type.append("robotaxi")
    scenerio_5_robotaxi_horizon.append(scenerio_5_amount_robotaxi)
    scenerio_5_youtube_horizon.append(0)
    scenerio_5_amount_robotaxi = scenerio_5_amount_robotaxi + robotaxi_increase
  else:
    scenerio_5_campaign_type.append("none")
    scenerio_5_robotaxi_horizon.append(0)
    scenerio_5_youtube_horizon.append(0)

  # 6. Split the budget  25% /  75%   YouTube Shorts / RoboTaxi (5 / 15 days)
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 15):
    scenerio_6_campaign_type.append("robotaxi")
    scenerio_6_robotaxi_horizon.append(scenerio_6_amount_robotaxi)
    scenerio_6_youtube_horizon.append(0)
    scenerio_6_amount_robotaxi = scenerio_6_amount_robotaxi + robotaxi_increase
  elif day >= date(2024, 12, 1) and day <= date(2024, 12, 5):
    scenerio_6_campaign_type.append("youtube_shorts")
    scenerio_6_robotaxi_horizon.append(0)
    scenerio_6_youtube_horizon.append(scenerio_6_amount_youtube)
    scenerio_6_amount_youtube = scenerio_6_amount_youtube + youtube_increase
  else:
    scenerio_6_campaign_type.append("none")
    scenerio_6_robotaxi_horizon.append(0)
    scenerio_6_youtube_horizon.append(0)

  # 7. Split the budget  50% /  50%   YouTube Shorts / RoboTaxi (10 / 10 days)
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 10):
    scenerio_7_campaign_type.append("robotaxi")
    scenerio_7_robotaxi_horizon.append(scenerio_7_amount_robotaxi)
    scenerio_7_youtube_horizon.append(0)
    scenerio_7_amount_robotaxi = scenerio_7_amount_robotaxi + robotaxi_increase
  elif day >= date(2024, 12, 1) and day <= date(2024, 12, 10):
    scenerio_7_campaign_type.append("youtube_shorts")
    scenerio_7_robotaxi_horizon.append(0)
    scenerio_7_youtube_horizon.append(scenerio_7_amount_youtube)
    scenerio_7_amount_youtube = scenerio_7_amount_youtube + youtube_increase
  else:
    scenerio_7_campaign_type.append("none")
    scenerio_7_robotaxi_horizon.append(0)
    scenerio_7_youtube_horizon.append(0)

  # 8. Split the budget  75% /  25%   YouTube Shorts / RoboTaxi (15 / 5 days)
  if day >= date(2024, 11, 1) and day <= date(2024, 11, 5):
    scenerio_8_campaign_type.append("robotaxi")
    scenerio_8_robotaxi_horizon.append(scenerio_8_amount_robotaxi)
    scenerio_8_youtube_horizon.append(0)
    scenerio_8_amount_robotaxi = scenerio_8_amount_robotaxi + robotaxi_increase
  elif day >= date(2024, 12, 1) and day <= date(2024, 12, 15):
    scenerio_8_campaign_type.append("youtube_shorts")
    scenerio_8_robotaxi_horizon.append(0)
    scenerio_8_youtube_horizon.append(scenerio_8_amount_youtube)
    scenerio_8_amount_youtube = scenerio_8_amount_youtube + youtube_increase
  else:
    scenerio_8_campaign_type.append("none")
    scenerio_8_robotaxi_horizon.append(0)
    scenerio_8_youtube_horizon.append(0)



In [ ]:
# This is our sales data
inputs = [context_sales_input, context_sales_input, context_sales_input, context_sales_input, context_sales_input, context_sales_input, context_sales_input, context_sales_input, context_sales_input]

# These are our categorical covariates (additional factors that we think might influence the thing we're trying to predict).
# Here we consider the type of campaign that is being run
dynamic_categorical_covariates = {
    "campaign_type" : [scenerio_0_campaign_type, scenerio_1_campaign_type, scenerio_2_campaign_type, scenerio_3_campaign_type, scenerio_4_campaign_type, scenerio_5_campaign_type, scenerio_6_campaign_type, scenerio_7_campaign_type, scenerio_8_campaign_type]
}

# These are our numerical covariates (additional numeric factors, just like the categories, but numbers)
# Here we consider the budget (cummlative spend) of our campaigns
dynamic_numerical_covariates = {
    "robotaxi_cumulative_campaign_spend" : [scenerio_0_robotaxi_horizon, scenerio_1_robotaxi_horizon, scenerio_2_robotaxi_horizon, scenerio_3_robotaxi_horizon, scenerio_4_robotaxi_horizon, scenerio_5_robotaxi_horizon, scenerio_6_robotaxi_horizon, scenerio_7_robotaxi_horizon, scenerio_8_robotaxi_horizon],
    "youtube_shorts_cumulative_campaign_spend" : [scenerio_0_youtube_horizon, scenerio_1_youtube_horizon, scenerio_2_youtube_horizon, scenerio_3_youtube_horizon, scenerio_4_youtube_horizon, scenerio_5_youtube_horizon, scenerio_6_youtube_horizon, scenerio_7_youtube_horizon, scenerio_8_youtube_horizon]
}

# These are our static covariates (additional factors that we think are fixed, like the price of a product)
static_numerical_covariates = {
}

# These are our static categorical covariates (additional factors that we think are fixed, like a menu item)
static_categorical_covariates = {
}

# frequency of each context time series. 0 for high frequency (default), 1 for medium, and 2 for low.
frequency = [0, 0, 0, 0, 0, 0, 0, 0, 0]

model_forecast, xreg_forecast = model.forecast_with_covariates(
    inputs=inputs,
    dynamic_categorical_covariates=dynamic_categorical_covariates,
    dynamic_numerical_covariates=dynamic_numerical_covariates,
    static_numerical_covariates=static_numerical_covariates,
    static_categorical_covariates=static_categorical_covariates,
    freq=frequency,
    xreg_mode="xreg + timesfm",              # default
    ridge=0.0,
    force_on_cpu=False,
    normalize_xreg_target_per_input=True,    # default
)

In [ ]:
scenerio_0_sum = sum(model_forecast[0])
scenerio_1_sum = sum(model_forecast[1])
scenerio_2_sum = sum(model_forecast[2])
scenerio_3_sum = sum(model_forecast[3])
scenerio_4_sum = sum(model_forecast[4])
scenerio_5_sum = sum(model_forecast[5])
scenerio_6_sum = sum(model_forecast[6])
scenerio_7_sum = sum(model_forecast[7])
scenerio_8_sum = sum(model_forecast[8])

print (f"Scenerio 0: Baseline - No campaign: {scenerio_0_sum:,.2f}")
print (f"Scenerio 1: Split the budget   0% / 100%   RoboTaxi / YouTube Shorts: {scenerio_1_sum:,.2f}")
print (f"Scenerio 2: Split the budget  25% /  75%   RoboTaxi / YouTube Shorts: {scenerio_2_sum:,.2f}")
print (f"Scenerio 3: Split the budget  50% /  50%   RoboTaxi / YouTube Shorts: {scenerio_3_sum:,.2f}")
print (f"Scenerio 4: Split the budget  75% /  25%   RoboTaxi / YouTube Shorts: {scenerio_4_sum:,.2f}")
print (f"Scenerio 5: Split the budget   0% / 100%   YouTube Shorts / RoboTaxi: {scenerio_5_sum:,.2f}")
print (f"Scenerio 6: Split the budget  25% /  75%   YouTube Shorts / RoboTaxi: {scenerio_6_sum:,.2f}")
print (f"Scenerio 7: Split the budget  50% /  50%   YouTube Shorts / RoboTaxi: {scenerio_7_sum:,.2f}")
print (f"Scenerio 8: Split the budget  75% /  25%   YouTube Shorts / RoboTaxi: {scenerio_8_sum:,.2f}")

In [ ]:
import matplotlib.pyplot as plt

"""
0. Baseline - No campaign
1. Split the budget   0% / 100%   RoboTaxi / YouTube Shorts
2. Split the budget  25% /  75%   RoboTaxi / YouTube Shorts
3. Split the budget  50% /  50%   RoboTaxi / YouTube Shorts
4. Split the budget  75% /  25%   RoboTaxi / YouTube Shorts
5. Split the budget   0% / 100%   YouTube Shorts / RoboTaxi
6. Split the budget  25% /  75%   YouTube Shorts / RoboTaxi
7. Split the budget  50% /  50%   YouTube Shorts / RoboTaxi
8. Split the budget  75% /  25%   YouTube Shorts / RoboTaxi
"""
scenerio_0 = model_forecast[0]
scenerio_2 = model_forecast[2]
scenerio_8 = model_forecast[8]

plt.figure(figsize=(14, 6))

# No Campaign
plt.plot(list_predicted_dates, scenerio_0, label='No Campaign', marker='s', linestyle='-.', color='#{:02x}{:02x}{:02x}'.format(92, 200, 243))

plt.plot(list_predicted_dates, scenerio_2, label='25% / 75%   RoboTaxi / YouTube', marker='^', linestyle='--', color='#{:02x}{:02x}{:02x}'.format(53, 106, 228))

# Set labels and title
plt.xlabel('Date')
plt.ylabel('Predicted Sales')
plt.title('Predicted Sales Comparison')

# Legend
plt.legend()

# Rotate x-axis labels for better readability (if necessary)
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()

#### Forecast Data (Setup)

In [ ]:
# predict the next 2 months "Nov/Dev 2024"
from datetime import date, timedelta
start_date = date(2024, 11, 1)
end_date = date(2024, 12, 31)
delta = end_date - start_date
list_predicted_dates = []

robotaxi_cumulative_campaign_spend = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0]
youtube_shorts_cumulative_campaign_spend =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
existing_sales_input = [2300.00,2302.30,2306.90,2306.90,2318.44,2327.71,2332.37,2332.37,2344.03,2351.06,2355.76,2365.19,2374.65,2386.52,2398.45,2528.45,2591.24,2664.76,2738.95,2799.81,2872.12,2933.23,3009.31,3562.04,3695.93,3834.40,3714.42,3466.79,3225.60,2986.41,2737.54,2715.37,2703.91,2684.23,2686.92,2521.18,2528.75,2538.86,2541.40,2546.48,2559.21,2559.21,2559.21,2569.45,2574.59,2577.16,2584.90,2592.65,2592.65,2595.24,2608.22,2671.03,2749.58,2830.97,2921.25,3003.20,3082.23,3173.91,3253.26,3332.61,3418.78,3505.28,3595.70,3679.28,3774.30,3866.17,3958.56,4047.42,4144.88,4247.29,4341.95,2741.71,2744.45,2744.45,2749.94,2758.19,2763.71,2774.76,2785.86,2794.22,2802.60,2805.40,2805.40,2813.82,2827.89,2827.89,2833.55,2847.71,2859.10,2861.96,2861.96,2867.69,2870.55,2882.04,2884.92,2899.34,2910.94,2913.85,2981.05,3068.72,3156.40,3250.57,3351.77,3453.74,3549.38,3649.03,3752.98,3857.71,3947.42,4037.14,4139.23,4237.67,4340.82,4453.41,4562.47,4663.03,4754.46,4870.12,3072.16,3081.38,3093.70,3106.08,3112.29,3112.29,3127.85,3130.98,3134.11,3143.51,3143.51,3152.94,3152.94,3317.21,3402.98,3482.12,3571.94,3654.97,3753.10,3832.95,3920.63,4654.67,4834.44,5020.56,4878.04,4566.50,4261.52,3937.65,3616.73,3594.60,3561.62,3539.23,3549.85,3330.89,3334.22,3340.89,3347.57,3347.57,3354.26,3360.97,3371.05,3377.80,3384.55,3394.71,3404.89,3415.10,3418.52,3418.52,3418.52,3435.61,3449.35,3459.70,3463.16,3470.09,3483.97,3501.39,3511.89,3522.43,3533.00,3540.06,3550.68,3564.88,3564.88,3564.88,3564.88,3572.01,3582.73,3672.66,3792.02,3900.36,4008.70,4125.28,4250.78,4377.21,4500.10,4619.08,4743.25,4868.11,4988.71,5114.87,5236.51,5347.93,5475.72,5598.65,5739.17,5857.56,5994.08,3773.63,3792.50,3807.67,3822.90,3834.37,3842.04,3861.25,3865.11,3880.57,3892.21,3896.11,3915.59,3923.42,3943.03,3958.81,3970.68,3986.57,4006.50,4010.51,4022.54,4042.65,4062.86,4083.18,4095.43,4111.81,4115.92,4132.38,4153.05,4157.20,4169.67,4173.84,4190.54,4207.30,4207.30,4228.33,4232.56,4236.79,4245.27,4266.49,4270.76,4279.30,4292.14,4524.77,4632.51,4754.46,4872.24,4980.51,5088.79,5207.45,5326.57,6330.14,6548.42,6787.00,6568.07,6130.20,5703.71,5275.49,4860.05,4830.31,4790.78,4751.17,4770.18,4475.94,4493.85,4507.33,4529.86,4552.51,4575.28,4584.43,4593.59,4593.59,4616.56,4630.41,4635.04,4653.58,4658.24,4658.24,4667.55,4686.22,4704.97,4728.49,4737.95,4832.71,4979.82,5122.10,5274.91,5439.15,5604.61,5771.31,5927.43,6084.14,6235.23,6405.76,6564.45,6710.32,6890.48,7072.27,7234.05,7418.59,7597.23,7761.69,7934.69,5000.35,5010.35,5035.40,5055.54,5075.77,5085.92,5096.09,5372.30,5511.21,5639.38,5773.31,5919.31,6072.19,6226.19,6387.68,7591.17,7876.49,8171.60,7923.81,7395.56,6887.91,6364.43,5863.23,5821.55,5779.67,5749.06,5766.31,5416.02,5416.02,5426.85,5432.28,5459.44,5486.74,5514.17,5530.71,5558.37,5575.04,5586.19,5591.78,5608.55,5608.55,5630.99,5630.99,5647.88,5647.88,5653.53,5676.14,5687.50,5715.93,5738.80,5738.80,5767.49,5796.33,5813.72,5813.72,5842.79,5842.79,5860.31,5866.17,5872.04,5883.78,5901.44,5930.94,5936.87,5960.62,5984.46,5996.43,6008.43,6026.45,6346.76,6510.87,6662.28,6834.14,7020.94,7180.74,7362.86,7553.83,8985.98,9295.84,9634.51,9361.02,8780.64,8161.60,7563.92,6947.46,6898.07,6834.79,6805.36,6812.16,6366.51,6366.51,6391.97,6411.15,6430.38,6462.54,6481.92,6514.33,6527.36,6546.94,6573.13,6586.28,6599.45,6632.45,6652.34,6659.00,6678.97,6685.65,6692.34,6692.34,6725.80,6732.53,6759.46,6779.73,6786.51,6943.01,7175.81,7388.21,7631.41,7869.01,8100.32,8316.33,8524.23,8758.34,9002.74,9239.74,9478.08,9708.09,9919.13,10150.44,10403.35,10626.28,10860.48,11073.43,11308.96,7141.00,7155.28,7191.06,7219.82,7219.82,7241.48,7241.48,7241.48,7241.48,7277.69,7306.80,7306.80,7321.42,7321.42,7343.38,7350.72,7372.78,7409.64,7819.02,8005.19,8224.12,8419.45,8649.58,8872.96,9107.06,9343.27,11092.56,11520.96,11928.80,11590.18,10849.95,10125.33,9393.19,8636.26,8557.75,8479.24,8417.53,8451.20,7906.21,7937.84,7953.71,7969.62,7985.56,8001.53,8009.53,8049.58,8065.68]
campaign_type = []

for i in range(len(existing_sales_input)):
  if robotaxi_cumulative_campaign_spend[i] > 0:
    campaign_type.append("robotaxi")
  elif youtube_shorts_cumulative_campaign_spend[i] == 0:
    campaign_type.append("youtube_shorts")
  else:
    campaign_type.append("none")

campaign_type_no_campaign = campaign_type.copy()

robotaxi_campaign_no_budget = robotaxi_cumulative_campaign_spend.copy()
youtube_shorts_no_budget = youtube_shorts_cumulative_campaign_spend.copy()

print("Existing Sizes")
print(f"len(existing_sales_input): {len(existing_sales_input)}")
print(f"len(robotaxi_cumulative_campaign_spend): {len(robotaxi_cumulative_campaign_spend)}")
print(f"len(youtube_shorts_cumulative_campaign_spend): {len(youtube_shorts_cumulative_campaign_spend)}")
print(f"len(campaign_type): {len(campaign_type)}")
print(f"len(campaign_type_no_campaign): {len(campaign_type_no_campaign)}")
print(f"len(list_predicted_dates): {len(list_predicted_dates)}")

cumulative_robotaxi = 100
cumulative_youtube_shorts = 100

for i in range(delta.days + 1):
  campaign_string = "none"
  day = start_date + timedelta(days=i)
  list_predicted_dates.append(day)

  if day >= date(2024, 11, 1) and day <= date(2024, 11, 10):
    robotaxi_cumulative_campaign_spend.append(cumulative_robotaxi)
    cumulative_robotaxi = cumulative_robotaxi + robotaxi_increase
    campaign = True
    campaign_string = "robotaxi"
  else:
    robotaxi_cumulative_campaign_spend.append(0)
    cumulative_robotaxi = 100

  if day >= date(2024, 12, 1) and day <= date(2024, 12, 30):
    youtube_shorts_cumulative_campaign_spend.append(cumulative_youtube_shorts)
    cumulative_youtube_shorts = cumulative_youtube_shorts + 100
    campaign = True
    campaign_string = "youtube_shorts"
  else:
    youtube_shorts_cumulative_campaign_spend.append(0)
    cumulative_youtube_shorts = 100

  campaign_type.append(campaign_string)
  campaign_type_no_campaign.append("none")

  robotaxi_campaign_no_budget.append(0)
  youtube_shorts_no_budget.append(0)

print("")
print("")
print("Added Nov/Dev 2024")
print(f"len(existing_sales_input): {len(existing_sales_input)}")
print(f"len(robotaxi_cumulative_campaign_spend): {len(robotaxi_cumulative_campaign_spend)}")
print(f"len(youtube_shorts_cumulative_campaign_spend): {len(youtube_shorts_cumulative_campaign_spend)}")
print(f"len(campaign_type): {len(campaign_type)}")
print(f"len(campaign_type_no_campaign): {len(campaign_type_no_campaign)}")
print(f"len(list_predicted_dates): {len(list_predicted_dates)}")

#### No Campaign Forecast

In [ ]:
# This is our sales data
inputs = [existing_sales_input]

# These are our categorical covariates (additional factors that we think might influence the thing we're trying to predict).
# Here we consider the type of campaign that is being run
dynamic_categorical_covariates = {
    "campaign_type" : [campaign_type_no_campaign]
}

# These are our numerical covariates (additional numeric factors, just like the categories, but numbers)
# Here we consider the budget (cummlative spend) of our campaigns
dynamic_numerical_covariates = {
    "robotaxi_cumulative_campaign_spend" : [robotaxi_campaign_no_budget],
    "youtube_shorts_cumulative_campaign_spend" : [youtube_shorts_no_budget]
}

# These are our static covariates (additional factors that we think are fixed, like the price of a product)
static_numerical_covariates = {
}

# These are our static categorical covariates (additional factors that we think are fixed, like a menu item)
static_categorical_covariates = {
}

# frequency of each context time series. 0 for high frequency (default), 1 for medium, and 2 for low.
frequency = [0]

model_forecast_no_campaign, xreg_forecast_no_campaign = model.forecast_with_covariates(
    inputs=inputs,
    dynamic_categorical_covariates=dynamic_categorical_covariates,
    dynamic_numerical_covariates=dynamic_numerical_covariates,
    static_numerical_covariates=static_numerical_covariates,
    static_categorical_covariates=static_categorical_covariates,
    freq=frequency,
    xreg_mode="xreg + timesfm",              # default
    ridge=0.0,
    force_on_cpu=False,
    normalize_xreg_target_per_input=True,    # default
)

#### Campaign Forecast

In [ ]:
# This is our sales data
inputs = [existing_sales_input]

# These are our categorical covariates (additional factors that we think might influence the thing we're trying to predict).
# Here we consider the type of campaign that is being run
dynamic_categorical_covariates = {
     "campaign_type" : [campaign_type]
}

# These are our numerical covariates (additional numeric factors, just like the categories, but numbers)
# Here we consider the budget (cummlative spend) of our campaigns
dynamic_numerical_covariates = {
    "robotaxi_cumulative_campaign_spend" : [robotaxi_cumulative_campaign_spend],
    "youtube_shorts_cumulative_campaign_spend" : [youtube_shorts_cumulative_campaign_spend]
}

# These are our static covariates (additional factors that we think are fixed, like the price of a product)
static_numerical_covariates = {
}

# These are our static categorical covariates (additional factors that we think are fixed, like a menu item)
static_categorical_covariates = {
}

# frequency of each context time series. 0 for high frequency (default), 1 for medium, and 2 for low.
frequency = [0]

model_forecast_with_campaign, xreg_forecast_with_campaign = model.forecast_with_covariates(
    inputs=inputs,
    dynamic_categorical_covariates=dynamic_categorical_covariates,
    dynamic_numerical_covariates=dynamic_numerical_covariates,
    static_numerical_covariates=static_numerical_covariates,
    static_categorical_covariates=static_categorical_covariates,
    freq=frequency,
    xreg_mode="xreg + timesfm",              # default
    ridge=0.0,
    force_on_cpu=False,
    normalize_xreg_target_per_input=True,    # default
)

#### Chart the Results

In [ ]:
# prompt: sum each value of model_forecast_no_campaign[0]

no_campaign_sum = sum(model_forecast_no_campaign[0])
campaign_sum = sum(model_forecast_with_campaign[0])

print (f"no_campaign_sum: {no_campaign_sum:,.2f}")
print (f"campaign_sum:    {campaign_sum:,.2f}")
print (f"difference:      {campaign_sum - no_campaign_sum:,.2f}")

In [ ]:
# 120,208.92

In [ ]:
model_forecast_no_campaign[0]

In [ ]:
model_forecast_with_campaign[0]

In [ ]:
import matplotlib.pyplot as plt

forecast_no_campaign = model_forecast_no_campaign[0]
forecast_with_campaign = model_forecast_with_campaign[0]

plt.figure(figsize=(14, 6))

# No Campaign
plt.plot(list_predicted_dates, forecast_no_campaign, label='No Campaign', marker='s', linestyle='-.', color='#{:02x}{:02x}{:02x}'.format(92, 200, 243))

# With Campaign
plt.plot(list_predicted_dates, forecast_with_campaign, label='With Campaign', marker='^', linestyle='--', color='#{:02x}{:02x}{:02x}'.format(53, 106, 228))

# Set labels and title
plt.xlabel('Date')
plt.ylabel('Predicted Sales')
plt.title('Predicted Sales Comparison')

# Legend
plt.legend()

# Rotate x-axis labels for better readability (if necessary)
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

forecast_no_campaign = model_forecast_no_campaign[0]
forecast_with_campaign = model_forecast_with_campaign[0]

percent_increase = []

for i in range(len(forecast_no_campaign)):
    percent_increase.append((forecast_with_campaign[i] - forecast_no_campaign[i]) / forecast_no_campaign[i])

plt.figure(figsize=(14, 6))

# Sales Percent Increase
plt.plot(list_predicted_dates, percent_increase, label='Percent Increase', marker='s', linestyle='-.', color='#{:02x}{:02x}{:02x}'.format(92, 200, 243))


# Set labels and title
plt.xlabel('Date')
plt.ylabel('Predicted Sales')
plt.title('Predicted Sales Comparison')

# Legend
plt.legend()

# Rotate x-axis labels for better readability (if necessary)
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()

### Clean Up

In [ ]:
# Placeholder

### Reference Links


- [Google.com](https://www.google.com)

In [ ]:
import matplotlib.pyplot as plt

# Define the parameters for the profit curve
max_profit = 10000  # Maximum profit achievable
plateau_point = 1500  # Ad spend at which profit plateaus

# Define functions for ad spend and profit
def ad_spend_curve(x):
  return x

def profit_curve(x):
  if x <= plateau_point:
    return max_profit * (x / plateau_point)  # Linear increase until plateau
  else:
    return max_profit  # Plateau after plateau point

# Generate data for ad spend and profit
ad_spends = range(0, 2001, 100)
profits = [profit_curve(x) for x in ad_spends]

# Plot the curves
plt.plot(ad_spends, profits, label="Profit")
plt.plot(ad_spends, ad_spend_curve(ad_spends), label="Ad Spend")

# Set labels and title
plt.xlabel("Ad Spend ($)")
plt.ylabel("Profit ($)")
plt.title("Ad Spend vs. Profit")

# Add legend
plt.legend()

# Show the plot
plt.show()